# Tools and Functions

In [2]:
// IMPORTS

#include <iostream> 
#include <random>
#include <sstream>
#include <string>
#include <chrono>   

In [3]:
// VALUE PRINT

template <typename T>
void vprint(const T& x) {
    std::cout << x << std::endl;
}

In [4]:
// ARRAY PRINT

template <typename T>
void aprint(const T* array, size_t size) {
    for (size_t i = 0; i < size; ++i) {
        std::cout << array[i] << " ";
    }
    std::cout << std::endl; 
}

In [5]:
// MATRIX PRINT (ONLY FOR INTEGER VALUES)

void mprint(int** matrix, size_t rows, size_t cols) {
    for (size_t i = 0; i < rows; ++i) {
        for (size_t j = 0; j < cols; ++j) {
            std::cout << matrix[i][j] << " ";
        }
        std::cout << std::endl;
    }
    std::cout << std::endl;
}

In [6]:
// RANDOM SETUP
// from 1 to 9

std::random_device rd; // Obtain a random number from hardware
std::mt19937 gen(rd()); // Seed the generator
std::uniform_int_distribution<> int_dis(1, 9);

In [7]:
// RANDOM GEN

int randInt(){
    return int_dis(gen);
}

In [8]:
// TIME

class Timer {
public:
    void start() {
        startTimepoint = std::chrono::high_resolution_clock::now();
    }

    void end(const std::string& unit = "milli") {
        auto endTimepoint = std::chrono::high_resolution_clock::now();

        if (unit == "micro") {
            auto duration = std::chrono::duration_cast<std::chrono::microseconds>(endTimepoint - startTimepoint);
            std::cout << "Time taken by function: " << duration.count() << " microseconds" << std::endl;
        } else if (unit == "milli") {
            auto duration = std::chrono::duration_cast<std::chrono::milliseconds>(endTimepoint - startTimepoint);
            std::cout << std::fixed << std::setprecision(3);
            std::cout << "Time taken by function: " << duration.count() << " milliseconds" << std::endl;
        } else if (unit == "sec") {
            auto duration = std::chrono::duration<double>(endTimepoint - startTimepoint);
            std::cout << std::fixed << std::setprecision(3);
            std::cout << "Time taken by function: " << duration.count() << " seconds" << std::endl;
        } else {
            std::cout << "Invalid time unit. Use 'micro', 'milli', or 'sec'." << std::endl;
        }
    }

private:
    std::chrono::time_point<std::chrono::high_resolution_clock> startTimepoint;
};

Timer timer;


# Cache Basics

Get some information about the CPU on the current system (in that case from the codespace).

In [9]:
! lscpu

Architecture:                       x86_64
CPU op-mode(s):                     32-bit, 64-bit
Byte Order:                         Little Endian
Address sizes:                      48 bits physical, 48 bits virtual
CPU(s):                             2
On-line CPU(s) list:                0,1
Thread(s) per core:                 2
Core(s) per socket:                 1
Socket(s):                          1
NUMA node(s):                       1
Vendor ID:                          AuthenticAMD
CPU family:                         25
Model:                              1
Model name:                         AMD EPYC 7763 64-Core Processor
Stepping:                           1
CPU MHz:                            3243.159
BogoMIPS:                           4890.85
Virtualization:                     AMD-V
Hypervisor vendor:                  Microsoft
Virtualization type:                full
L1d cache:                          32 KiB
L1i cache:                          32 KiB
L2 cache:           

Lets look at the cache by its own.

In [10]:
! lscpu | grep -i "cache"

L1d cache:                          32 KiB
L1i cache:                          32 KiB
L2 cache:                           512 KiB
L3 cache:                           32 MiB


We have 2 CPUs available with 32 KiB of L1 cache (sometimes also known as level 1 cache).

The L1 cache is a type of CPU cache that is closest to the processor cores. It is typically split into two distinct caches: L1i (L1 instruction cache) and L1d (L1 data cache). 

Purpose:

- L1i Cache (Instruction Cache):
    - Stores instructions that the CPU needs to execute.
    - Optimizes the fetching of instructions from memory to the CPU.
    - Improves the performance of the instruction pipeline by reducing the time it takes to fetch instructions.

- L1d Cache (Data Cache):
    - Stores data that the CPU needs to process.
    - Optimizes the fetching of data from memory to the CPU.
    - Enhances the performance of data retrieval operations.The L1 cache, or Level 1 cache, is a type of CPU cache that is closest to the processor cores. It is typically split into two distinct caches: L1i (L1 instruction cache) and L1d (L1 data cache). Here are the differences between them:

We also have 512 KiB of L2 and 32 MiB of L3 cache.

Let look at the cache line size.

In [11]:
! getconf LEVEL1_DCACHE_LINESIZE

64


In [12]:
! getconf LEVEL1_ICACHE_LINESIZE

64


In [13]:
! getconf LEVEL2_CACHE_LINESIZE

64


In [14]:
! getconf LEVEL3_CACHE_LINESIZE

64


The cache line size is 64 bytes.

This means, with a L1 cache size of 32 KiB and a cache line size of 64 bytes we have 500 cache lines.
When storing loading data in the L1 cache, we load a whole cache line, so in that case 64 bytes per load.

# Cpp Basics

Get the size of an integer in byte.

In [15]:
vprint(sizeof(int));

4


This means that we use 4 bytes (**32 bits**) per integer, so $−2^{31}$ to $231^{−1231}−1$ for signed integers.

## Vectors / Arrays with malloc

void* malloc(size_t size);

### How malloc Works

- Requesting Memory: When malloc is called, it requests a block of memory from the heap (**RAM**) of the specified size.
- Allocating Memory: The heap manager in the runtime library tries to find a suitable **contiguous** block of free memory that meets the requested size. If it finds such a block, it marks it as used and returns a **pointer** to the beginning of the block.
- Return Value: If successful, malloc returns a void* pointer to the beginning of the allocated memory block. If it fails (e.g., if there is not enough memory available), it returns NULL.

Lets make an array with 10 integer values.
(Note: We have to keep track of the size and / or type of the array to iterate over it.)

In [16]:
int* arr = (int*)malloc(10 * sizeof(int));

We allocate 10 * 4 bytes with malloc and cast the void pointer to an integer pointer that points to the first element.
This integer pointer must be stored in an integer pointer variable, in that case "arr". 

Lets set the value at positon [0] to 1.

In [17]:
arr[0] = 1;

And print it.

In [18]:
vprint(arr[0]);

1


Lets quickly write a function to fill the array with random integer values.
(The array must be an integer array.)

In [19]:
void fillRandom(int* arr, size_t size) {

    for(int i = 0; i < size; i++){
        arr[i] = randInt();
    }

}

And a function that to fill the array with zeros.

In [20]:
void fillZeros(int* arr, size_t size) {

    for(int i = 0; i < size; i++){
        arr[i] = 0;
    }

}

Lets fill the array with values.

In [21]:
fillRandom(arr, 10);

And print it.

In [22]:
aprint(arr, 10);

5 1 4 3 7 8 6 5 9 3 


## Matrices

We want to create a matrix of size n x m or rows x cols.

In [23]:
size_t rows = 4;
size_t cols = 4; 

A matrix is a set of pointers that are pointing to set of values (vecotrs) each.
Thats why we need to create a set of integer pointers first. 

In [24]:
int** matrix = (int**)malloc(rows * sizeof(int*));

Lets fill our now pointers by allocating the space needed.

In [25]:
for (int i = 0; i < rows; ++i) {
    
    matrix[i] = (int*)malloc(cols * sizeof(int));

}

To fill it, we must create a new function.

(Note: We could use the fillRandom function too.)

In [26]:
void fillMatrixRandom(int** matrix, size_t rows, size_t cols){

    for(int i = 0; i < rows; i++){
        
        for(int j = 0; j < cols; j++){
            
            matrix[i][j] = randInt();

        }
    
    }

}

And fill our matrix.

In [27]:
fillMatrixRandom(matrix, rows, cols);

Now we can print it.

In [28]:
mprint(matrix, rows, cols);

4 7 6 8 
6 7 7 5 
6 2 2 5 
4 3 3 9 



Lets clean everything up and make some functions. 

In [29]:
int* createVector(size_t size){
    int* v = (int*)malloc(size * sizeof(int));
    fillRandom(v, size);
    return v;
}

In [30]:
int* createZeroVector(size_t size){
    int* v = (int*)malloc(size * sizeof(int));
    fillZeros(v, size);
    return v;
}

In [31]:
int** createMatrix(size_t rows, size_t cols){
    int** m = (int**)malloc(rows * sizeof(int*));
    for (size_t i = 0; i < rows; i++) {
        m[i] = createVector(cols);
    }

    return m;
}

In [32]:
int** createZeroMatrix(size_t rows, size_t cols){
    int** m = (int**)malloc(rows * sizeof(int*));
    for (size_t i = 0; i < rows; i++) {
        m[i] = createZeroVector(cols);
    }

    return m;
}

And test the functions.

In [33]:
size_t size = 4;

int** m1 = createMatrix(size, size);
int** m2 = createZeroMatrix(size, size);

mprint(m1, size, size);
mprint(m2, size, size);

8 3 7 3 
8 6 9 8 
9 4 1 7 
4 1 1 4 

0 0 0 0 
0 0 0 0 
0 0 0 0 
0 0 0 0 



## Transpose

Lastly, we write a function to transpose a matrix.

In [34]:
int** transpose(int** in, size_t rows, size_t cols){

    int** out = createMatrix(cols, rows);

    for(size_t row = 0; row < rows; row++){

        for(size_t col = 0; col < cols; col++){
    
            out[col][row] = in[row][col];
            
        }
    }
    
    return out;

}

And test the function.

In [35]:
size_t rows = 5;
size_t cols = 3;

int** in = createMatrix(rows, cols);
mprint(in, rows, cols);

int** trans = transpose(in, rows, cols);

// Note the change from rows / cols to cols / rows !!!
mprint(trans, cols, rows);

9 7 8 
1 5 3 
3 1 3 
4 3 5 
4 9 8 

9 1 3 4 4 
7 5 1 3 9 
8 3 3 5 8 



# Matrix Operations

Lets look in some ways to perform the dot product.

For now, we use vectors or matrices of size: $n * 1024$. We define some values first.

In [36]:
unsigned int ntest = 4;

unsigned int n1 = 1024 * 1;      std::cout << "1024 * 1  = " << n1 << " and n * n = " << n1 * n1 << " values per matrix " << std::endl;
unsigned int n4 = 1024 * 4;      std::cout << "1024 * 4  = " << n4 << " and n * n = " << n4 * n4 << " values per matrix " << std::endl;
unsigned int n8 = 1024 * 8;      std::cout << "1024 * 8  = " << n8 << " and n * n = " << n8 * n8 << " values per matrix " << std::endl;
unsigned int n16 = 1024 * 16;    std::cout << "1024 * 16 = " << n16 << " and n * n = " << n16 * n16 << " values per matrix " << std::endl;
unsigned int n32 = 1024 * 32;    std::cout << "1024 * 32 = " << n32 << " and n * n = " << n32 * n32 << " values per matrix " << std::endl;

// 1024 * 64 is the maximum size for an unsigned integer and cant be represented. So we use 1024 * 63.
unsigned int n63 = 1024 * 63;    std::cout << "1024 * 63 = " << n63 << " and n * n = " << n63 * n63 << " values per matrix " << std::endl;

1024 * 1  = 1024 and n * n = 1048576 values per matrix 
1024 * 4  = 4096 and n * n = 16777216 values per matrix 
1024 * 8  = 8192 and n * n = 67108864 values per matrix 
1024 * 16 = 16384 and n * n = 268435456 values per matrix 
1024 * 32 = 32768 and n * n = 1073741824 values per matrix 
1024 * 63 = 64512 and n * n = 4161798144 values per matrix 


## Serial matrix-vector multiplication

First we define the funciton.

In [37]:
int* smvp(int** m, int* v, size_t size){
    
    int* out = createZeroVector(size);

    for(int row = 0; row < size; row++){
        
        for(int col = 0; col < size; col++){
        
            out[row] += m[row][col] * v[col];
        
        }
    }
    
    return out;
}

And test it with the ntest value. Therefore, we define a matrix and a vecotr.

In [38]:
int n = ntest;

int** m = createMatrix(n, n);
mprint(m, n, n);

int* v = createVector(n);
aprint(v, n);

4 8 4 1 
6 4 6 7 
7 1 7 4 
5 5 9 8 

4 5 5 5 


Now we run the calculations and use the inbuild notebook time tracking.

In [39]:
int* out = smvp(m, v, n);

And print it.

In [40]:
aprint(out, n);

81 109 88 130 


Now we can run the function with diffrent sizes for n.

In [41]:
int n = n16;

int** m = createMatrix(n, n);
int* v = createVector(n);

In [42]:
int* out = smvp(m, v, n);

Sometimes we want to mesure the time more precise, so we use a fuction to mesure the time in diffrent units.

In [43]:
timer.start();
int* out = smvp(m, v, n);
timer.end(); // = timer.end("milli");

Time taken by function: 615 milliseconds


In [44]:
timer.start();
int* out = smvp(m, v, n);
timer.end("sec");

Time taken by function: 0.666 seconds


In [45]:
timer.start();
int* out = smvp(m, v, n);
timer.end("micro");

Time taken by function: 665118 microseconds


## Serial matrix-transposed-vector multiplication

We store the matrix in the normal way, but change the calculations to calculate the matrix-vector product if the matrix would be transposed.

To do this, we change m[row][col] to m[col][row].

In [46]:
int* smtvp(int** m, int* v, size_t size){
    
    int* out = createZeroVector(size);

    for(int row = 0; row < size; row++){
        
        for(int col = 0; col < size; col++){
        
            out[row] += m[col][row] * v[col];
        
        }
    }
    
    return out;
}

In [47]:
int n = n16;

int** m = createMatrix(n, n);
int* v = createVector(n);

In [48]:
timer.start();
int* out = smtvp(m, v, n);
timer.end(); 

Time taken by function: 1949 milliseconds


We observe, that the time for the matrix-transposed-vector product is about 3 times the matrix-vector product.
The is obvious, because for each calculation we have to load a new cache line, because we want for example allways the first element
of each row. Therefore, we have to load each row again and again.

## GEMM

For these calculations we should define some diffrent values for n.

In [62]:
unsigned int n100 = 100;      std::cout << "n = " << n100 << " and n * n = " << n100 * n100 << " values per matrix " << std::endl;
unsigned int n1000 = 1000;      std::cout << "n = " << n1000 << " and n * n = " << n1000 * n1000 << " values per matrix " << std::endl;
unsigned int n10000 = 10000;      std::cout << "n = " << n10000 << " and n * n = " << n10000 * n10000 << " values per matrix " << std::endl;
unsigned int n100000 = 100000;      std::cout << "n = " << n100000 << " and n * n = " << n100000 * n100000 << " values per matrix " << std::endl;


n = 100 and n * n = 10000 values per matrix 
n = 1000 and n * n = 1000000 values per matrix 
n = 10000 and n * n = 100000000 values per matrix 
n = 100000 and n * n = 1410065408 values per matrix 


In [49]:
void rowOrientedGEMM(int** X, int** Y, int** Z, size_t n) {
    
    for (int i = 0; i < n; ++i) {
        
        for (int j = 0; j < n; ++j) {
            
            for (int k = 0; k < n; ++k) {
                
                Z[i][j] += X[i][k] * Y[k][j];
            
            }
        }
    }
}

In [50]:
void columnOrientedGEMM(int** X, int** Y, int** Z, size_t n) {
    
    for (int i = 0; i < n; ++i) {
        
        for (int j = 0; j < n; ++j) {
            
            for (int k = 0; k < n; ++k) {
                
                Z[j][i] += X[j][k] * Y[k][i];
            
            }
        }
    }
}

In [72]:
// CHATBOT

void blockedGEMM(int** X, int** Y, int** Z, int n, int blockSize) {
    
    for (int i = 0; i < n; i += blockSize) {
        for (int j = 0; j < n; j += blockSize) {
            for (int k = 0; k < n; k += blockSize) {
    
                // Process sub-blocks
                for (int i1 = i; i1 < std::min(i + blockSize, n); ++i1) {
                    for (int j1 = j; j1 < std::min(j + blockSize, n); ++j1) {
                        for (int k1 = k; k1 < std::min(k + blockSize, n); ++k1) {
                            
                            Z[i1][j1] += X[i1][k1] * Y[k1][j1];
    
                        }
                    }
                }
            }
        }
    }
}

In [63]:
int n = n1000;

int** X = createMatrix(n, n);
int** Y = createMatrix(n, n);
int** Z = createMatrix(n, n);

In [64]:
timer.start();
rowOrientedGEMM(X, Y, Z, n);
timer.end("sec");

Time taken by function: 3.625 seconds


In [65]:
timer.start();
columnOrientedGEMM(X, Y, Z, n);
timer.end("sec");

Time taken by function: 3.401 seconds


In [68]:
int block_size = 4;

timer.start();
blockedGEMM(X, Y, Z, n, block_size);
timer.end("sec");

Time taken by function: 6.852 seconds


We observe, that the row and column based methodes are about equal.
The block based methode takes twice the time, but we tested it only for a block size of 4. 

In [78]:
int block_size = 66;

timer.start();
blockedGEMM(X, Y, Z, n, block_size);
timer.end("sec");

Time taken by function: 5.393 seconds


If we have a cache line size of 64 bytes and an intger value takes 4 bytes, we should get 16 integers in one cache line.

Therefore, we should cut the for example 1000 entries into about 64 blocks. But the effect is not as big as supposed. 

# Open MP

Using OpenMP, we can parallelize our programs. Unfortunately, this cannot be done well in the nodebook, so we run the code inside an external file.

First we create a header file with our tools and basic functions.

In [79]:
! cat src/Tools.hpp

#ifndef TOOLS_HPP
#define TOOLS_HPP

#include <iostream>
#include <random>
#include <sstream>
#include <string>
#include <chrono>
#include <cstddef>
#include <iomanip>

constexpr unsigned int ntest = 4;
constexpr unsigned int n1 = 1024 * 1;     
constexpr unsigned int n4 = 1024 * 4;      
constexpr unsigned int n8 = 1024 * 8;      
constexpr unsigned int n16 = 1024 * 16;    
constexpr unsigned int n32 = 1024 * 32;   
constexpr unsigned int n63 = 1024 * 63;  
constexpr unsigned int n100 = 100;      
constexpr unsigned int n1000 = 1000;     
constexpr unsigned int n10000 = 10000;     
constexpr unsigned int n100000 = 100000;  

// Printer class
class Printer {
public:
    // VALUE PRINT
    template <typename T>
    void value(const T& x);

    // ARRAY PRINT
    template <typename T>
    void array(const T* array, size_t size);

    // MATRIX PRINT (ONLY FOR INTEGER VALUES)
    void matrix(int** matrix, size_t rows, size_t cols);
};

// Timer class
class Timer {
public:
    void start()

And the implementation.

In [80]:
! cat src/Tools.cpp

#include "Tools.hpp"

void Printer::matrix(int** matrix, size_t rows, size_t cols) {
    for (size_t i = 0; i < rows; ++i) {
        for (size_t j = 0; j < cols; ++j) {
            std::cout << matrix[i][j] << " ";
        }
        std::cout << std::endl;
    }
    std::cout << std::endl;
}

template <typename T>
void Printer::value(const T& x) {
    std::cout << x << std::endl;
}

template <typename T>
void Printer::array(const T* array, size_t size) {
    for (size_t i = 0; i < size; ++i) {
        std::cout << array[i] << " ";
    }
    std::cout << std::endl;
}

// explicit template instantiations
template void Printer::value<int>(const int& x);
template void Printer::value<double>(const double& x);
template void Printer::array<int>(const int* array, size_t size);
template void Printer::array<double>(const double* array, size_t size);


// TIMER

void Timer::start() {
    startTimepoint = std::chrono::high_resolution_clock::now();
}

void Timer::end(const std::string& unit) {
    

Now we build a make file that takes our program that should be compiled as input.

In [81]:
! cat Makefile

# Variables
CXX = g++
CXXFLAGS = -O2 -Wall -fopenmp
SRC_DIR = src
BIN_DIR = bin
TOOLS_SRC = $(SRC_DIR)/Tools.cpp
TOOLS_OBJ = $(BIN_DIR)/Tools.o

# Create bin directory if it does not exist
$(shell mkdir -p $(BIN_DIR))

# Targets
all: $(BIN_DIR)/$(TARGET)

# Rule to compile the main .cpp file and link with Tools.o to create the executable
$(BIN_DIR)/$(TARGET): $(BIN_DIR)/$(TARGET).o $(TOOLS_OBJ)
	$(CXX) $(CXXFLAGS) -o $@ $^

# Rule to compile main .cpp file into object file
$(BIN_DIR)/%.o: $(SRC_DIR)/%.cpp
	$(CXX) $(CXXFLAGS) -c -o $@ $<

# Rule to compile Tools.cpp
$(TOOLS_OBJ): $(TOOLS_SRC)
	$(CXX) $(CXXFLAGS) -c -o $@ $<

# Rule to clean build artifacts
clean:
	rm -rf $(BIN_DIR)/*

.PHONY: all clean


Lastly, a litte helper script to compile and run the targeted code. 

In [99]:
! bash run.sh OpenMP_AXPY

Running make...
g++ -O2 -Wall -fopenmp -c -o bin/OpenMP_AXPY.o src/OpenMP_AXPY.cpp
g++ -O2 -Wall -fopenmp -o bin/OpenMP_AXPY bin/OpenMP_AXPY.o bin/Tools.o
Running ./bin/OpenMP_AXPY...
Time taken by function: 0.000 seconds
Time taken by function: 0.000 seconds


The code for OpenMP_AXPY is:

In [87]:
! cat src/OpenMP_AXPY.cpp

#include "Tools.hpp"

#include <omp.h>

void saxmpy(int a, int* x, int* y, size_t size){

    for(size_t i = 0; i < size; i++){

        y[i] += a * x[i];

    }

}

void paxmpy(int a, int* x, int* y, size_t size, int threads) {

    omp_set_num_threads(threads);

    #pragma omp parallel
    {
        #pragma omp for
        for (size_t i = 0; i < size; i++) {
            y[i] += a * x[i];
        }
    }
}

int main(){
    
    Timer time;
    Structur structur;
    // Printer printer;


    int* x = structur.createVector(n1000);
    int* y = structur.createVector(n1000);

    int a = 6;
    int threads = 2;

    time.start();
    saxmpy(a, x, y, n1000);
    time.end("sec");

    time.start();
    paxmpy(a, x, y, n1000, threads);
    time.end("sec");


}

